In [1]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths
import tensorflow as tf 
import tensorboard as tb 
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss,MAE,MAPE,RMSE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
import pandas as pd

# Load the excel file
data = pd.read_excel('C:/Users/xuyechi/Desktop/科研/tft/九寨沟客流量.xlsx')
# df["Date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
# df["date"] = pd.to_datetime(df["date"]).astype(int) // 10**9
data["year"] = data["year"].astype(str)
data["day"] = data["day"].astype(str)
data["month"] = data["month"].astype(str)
data['holiday'] = data['holiday'].astype(str)
data["tourist"] = data["tourist"].astype("float64")
data["Trend"]=data["Trend"].astype("float64")
data["Seasonal"]=data["Seasonal"].astype("float64")
data["Resid"]=data["Resid"].astype("float64")
data

,tourist,date,lag,time_idx,pc,mob,weekday,year,month,day,holiday,destination,Trend,Seasonal,Resid,seasonal,resid
0,279.0,2020-03-31,-1,1,367,858,Monday,2020,3,31,Normal,JiuZhaigou,290.691365,3970.854466,5017.454170,-29.145534,17.454170
1,208.0,2020-04-01,-1,2,388,856,Tuesday,2020,4,1,Normal,JiuZhaigou,318.851010,3934.201452,4954.947538,-65.798548,-45.052462
2,295.0,2020-04-02,-1,3,445,873,Wednesday,2020,4,2,Normal,JiuZhaigou,347.046695,3948.992652,4998.960653,-51.007348,-1.039347
3,311.0,2020-04-03,-1,4,333,877,Thursday,2020,4,3,Normal,JiuZhaigou,375.607632,3868.047896,5067.344472,-131.952104,67.344472
4,528.0,2020-04-04,-1,5,218,945,Friday,2020,4,4,Tomb_Sweeping_Day,JiuZhaigou,405.057652,4138.158943,4984.783405,138.158943,-15.216595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,18271.0,2022-08-16,-1,869,586,2795,Monday,2022,8,16,Normal,JiuZhaigou,17650.768622,4471.042606,5149.188772,471.042606,149.188772
869,19132.0,2022-08-17,-1,870,685,2871,Tuesday,2022,8,17,Normal,JiuZhaigou,17687.353387,7904.887545,2539.759068,3904.887545,-2460.240932
870,15209.0,2022-08-18,-1,871,2868,3588,Wednesday,2022,8,18,Normal,JiuZhaigou,17721.182972,2244.456347,4243.360682,-1755.543653,-756.639318
871,15902.0,2022-08-19,-1,872,1102,3233,Thursday,2022,8,19,Normal,JiuZhaigou,17752.081802,2100.969245,5048.948953,-1899.030755,48.948953


In [12]:
max_prediction_length = 1
max_encoder_length = 30
# training_cutoff = data["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= 783],
    #data.iloc[3925:training_cutoff,:],
    time_idx="time_idx",
    target="Seasonal",
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    time_varying_known_categoricals=["month","weekday","day","holiday"],
    time_varying_known_reals=["time_idx","pc","mob"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "Seasonal",
        
    ],
    group_ids=['destination'],
    target_normalizer=GroupNormalizer(
        groups=['destination'], transformation="softplus"),
     # 取了对数
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,

   
)

# # inverse transform
# forecast = predictor.predict(test_data)
# forecast_samples = forecast.samples
# forecast_samples = training.target_normalizer.inverse_transform(forecast_samples)

In [13]:
# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)
# from_dataset是生成具有不同底层数据但相同变量编码器和标量的数据集的函数
# training 代表validation所需要的参数； data代表validation从哪选；


# create dataloaders for model
batch_size = 128  # set this between 32 to 128


# check if GPU is available
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# move dataloaders to device


train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size*10 , num_workers=0)


In [14]:
import pickle
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters



# create study
study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path="optuna_test",
    n_trials=50,
    max_epochs=50,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 128),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=30),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False
)


# save study results - also we can resume tuning at a later point in time
with open("test_study.pkl", "wb") as fout:
    pickle.dump(study, fout)

# show best hyperparameters
print(study.best_trial.params)

[I 2023-04-02 20:43:24,190] A new study created in memory with name: no-name-ff787481-b797-4e76-86e7-90f7cc348fda
D:\anaconda3\envs\myenv1\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision

TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2023-04-02 20:45:48,561] Trial 7 pruned. Trial was pruned at epoch 1.
D:\anaconda3\envs\myenv1\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: Tru

HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2023-04-02 20:46:14,892] Trial 15 pruned. Trial was pruned at epoch 1.
D:\anaconda3\envs\myenv1\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IP

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2023-04-02 20:46:46,440] Trial 23 pruned. Trial was pruned at epoch 1.
D:\anaconda3\envs\myenv1\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precisi

D:\anaconda3\envs\myenv1\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2023-04-02 20:47:05,520] Trial 32 pruned. Trial was pruned at epoch 

TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
[I 2023-04-02 20:48:12,789] Trial 40 finished with value: 2050.01123046875 and parameters: {'gradient_clip_val': 0.3093780371241915, 'hidden_size': 32, 'dropout': 0.1473046987151529, 'hidden_continuous_size': 14, 'attention_head_size': 2, 'learning_rate': 0.07863411540189284}. Best is trial 1 with value: 1747.357421875.
D:\anaconda3\envs\myenv1\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connecto

TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2023-04-02 20:48:35,457] Trial 48 pruned. Trial was pruned at epoch 1.
D:\anaconda3\envs\myenv1\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: Tr

{'gradient_clip_val': 0.31330228284676426, 'hidden_size': 15, 'dropout': 0.2757684537904421, 'hidden_continuous_size': 10, 'attention_head_size': 2, 'learning_rate': 0.0011749674283127688}


In [ ]:
#1 
#{'gradient_clip_val': 0.058711482089361826, 'hidden_size': 14, 
#'dropout': 0.2722111720167587, 'hidden_continuous_size': 10, 'attention_head_size': 2, 'learning_rate': 0.09285323457031537}

#3 
# {'gradient_clip_val': 0.011011271624431383, 'hidden_size': 30, 
#  'dropout': 0.2954455811444877, 'hidden_continuous_size': 14, 'attention_head_size': 2, 'learning_rate': 0.08100063625658273}


# 7
# {'gradient_clip_val': 0.06965995703621122, 'hidden_size': 70, 
#  'dropout': 0.20063425125576373, 'hidden_continuous_size': 18, 'attention_head_size': 1, 'learning_rate': 0.0969849703729482}

#15
# {'gradient_clip_val': 0.05170506339745095, 'hidden_size': 43,
#  'dropout': 0.27510184522694925, 'hidden_continuous_size': 23, 'attention_head_size': 1, 'learning_rate': 0.003971445810850568}

#30
# {'gradient_clip_val': 0.11278609968246897, 'hidden_size': 61, 
#  'dropout': 0.17501843926436106, 'hidden_continuous_size': 8, 'attention_head_size': 4, 'learning_rate': 0.002323645204383734}

# 60
# {'gradient_clip_val': 0.08963621290452617, 'hidden_size': 90, 
#  'dropout': 0.24069089701689708, 'hidden_continuous_size': 22, 'attention_head_size': 2, 'learning_rate': 0.027940923926813188}

# 90
# {'gradient_clip_val': 0.01229433160487252, 'hidden_size': 99,
#  'dropout': 0.21930970181368742, 'hidden_continuous_size': 90, 'attention_head_size': 4, 'learning_rate': 0.0033330257773551033}

In [15]:
# 1天
# {'gradient_clip_val': 0.31330228284676426, 'hidden_size': 15, 
# 'dropout': 0.2757684537904421, 'hidden_continuous_size': 10, 'attention_head_size': 2, 'learning_rate': 0.0011749674283127688}
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_last=True,
    save_top_k=1,  # 仅保留验证集上误差最小的模型
    filename="best_model_{epoch}",  # 您可以根据需要自定义文件名格式
    dirpath="saved_models"
)

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    enable_model_summary=True,
    gradient_clip_val=0.31330228284676426,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
    logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.0011749674283127688,
    hidden_size=15,
    attention_head_size=2,
    dropout=0.2757684537904421,
    hidden_continuous_size=10,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")



D:\anaconda3\envs\myenv1\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 23.0k


In [195]:
# 3 天
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
#3 
# {'gradient_clip_val': 0.011011271624431383, 'hidden_size': 30, 
#  'dropout': 0.2954455811444877, 'hidden_continuous_size': 14, 'attention_head_size': 2, 'learning_rate': 0.08100063625658273}

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_last=True,
    save_top_k=1,  # 仅保留验证集上误差最小的模型
    filename="best_model_{epoch}",  # 您可以根据需要自定义文件名格式
    dirpath="saved_models"
)


early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard




trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    enable_model_summary=True,
    gradient_clip_val=0.011011271624431383,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
    logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.08100063625658273,
    hidden_size=30,
    attention_head_size=2,
    dropout=0.2954455811444877,
    hidden_continuous_size=14,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 69.1k


In [18]:
# 7
# {'gradient_clip_val': 0.06965995703621122, 'hidden_size': 70, 
#  'dropout': 0.20063425125576373, 'hidden_continuous_size': 18, 'attention_head_size': 1, 'learning_rate': 0.0969849703729482}

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_last=True,
    save_top_k=1,  # 仅保留验证集上误差最小的模型
    filename="best_model_{epoch}",  # 您可以根据需要自定义文件名格式
    dirpath="saved_models"
)


early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard




trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    enable_model_summary=True,
    gradient_clip_val=0.06965995703621122,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
    logger=logger,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.0269849703729482,
    hidden_size=70,
    attention_head_size=1,
    dropout=0.20063425125576373,
    hidden_continuous_size=18,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=8,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

D:\anaconda3\envs\myenv1\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 305.7k


In [40]:
#15
# {'gradient_clip_val': 0.05170506339745095, 'hidden_size': 43,
#  'dropout': 0.27510184522694925, 'hidden_continuous_size': 23, 'attention_head_size': 1, 'learning_rate': 0.003971445810850568}
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_last=True,
    save_top_k=1,  # 仅保留验证集上误差最小的模型
    filename="best_model_{epoch}",  # 您可以根据需要自定义文件名格式
    dirpath="saved_models"
)


early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard




trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    enable_model_summary=True,
    gradient_clip_val=0.05170506339745095,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
    logger=logger,
)

# 0.009971445810850568
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.003971445810850568,
    hidden_size=43,
    attention_head_size=1,
    dropout=0.27510184522694925,
    hidden_continuous_size=23,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=8,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

D:\anaconda3\envs\myenv1\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 143.8k


In [ ]:
#30
# {'gradient_clip_val': 0.11278609968246897, 'hidden_size': 61, 
#  'dropout': 0.17501843926436106, 'hidden_continuous_size': 8, 'attention_head_size': 4, 'learning_rate': 0.002323645204383734}
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_last=True,
    save_top_k=1,  # 仅保留验证集上误差最小的模型
    filename="best_model_{epoch}",  # 您可以根据需要自定义文件名格式
    dirpath="saved_models"
)


early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard




trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    enable_model_summary=True,
    gradient_clip_val=0.11278609968246897,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
    logger=logger,
)

# 0.009971445810850568
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.002323645204383734,
    hidden_size=61,
    attention_head_size=4,
    dropout=0.17501843926436106,
    hidden_continuous_size=8,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=8,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# 60
# {'gradient_clip_val': 0.08963621290452617, 'hidden_size': 90, 
#  'dropout': 0.24069089701689708, 'hidden_continuous_size': 22, 'attention_head_size': 2, 'learning_rate': 0.027940923926813188}
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_last=True,
    save_top_k=1,  # 仅保留验证集上误差最小的模型
    filename="best_model_{epoch}",  # 您可以根据需要自定义文件名格式
    dirpath="saved_models"
)


early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard



trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    enable_model_summary=True,
    gradient_clip_val=0.08963621290452617,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
    logger=logger,
)

# 0.009971445810850568
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.027940923926813188,
    hidden_size=90,
    attention_head_size=2,
    dropout=0.24069089701689708,
    hidden_continuous_size=22,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=8,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# 90
# {'gradient_clip_val': 0.01229433160487252, 'hidden_size': 99,
#  'dropout': 0.21930970181368742, 'hidden_continuous_size': 90, 'attention_head_size': 4, 'learning_rate': 0.0033330257773551033}
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_last=True,
    save_top_k=1,  # 仅保留验证集上误差最小的模型
    filename="best_model_{epoch}",  # 您可以根据需要自定义文件名格式
    dirpath="saved_models"
)


early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard



trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    enable_model_summary=True,
    gradient_clip_val=0.01229433160487252,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
    logger=logger,
)

# 0.009971445810850568
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.0033330257773551033,
    hidden_size=99,
    attention_head_size=4,
    dropout=0.21930970181368742,
    hidden_continuous_size=90,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=8,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 488   
3  | prescalers                         | ModuleDict                      | 160   
4  | static_variable_selection          | VariableSelectionNetwork        | 2.

Epoch 0:  86%|██████████████████████████▌    | 6/7 [00:01<00:00,  4.96it/s, loss=165, v_num=140, train_loss_step=157.0]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████████| 7/7 [00:01<00:00,  4.43it/s, loss=165, v_num=140, train_loss_step=157.0, val_loss=1.9e+3]
Epoch 1:  86%|▊| 6/7 [00:00<00:00,  6.72it/s, loss=166, v_num=140, train_loss_step=177.0, val_loss=1.9e+3, train_loss_e
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 1: 100%|█| 7/7 [00:01<00:00,  5.38it/s, loss=166, v_num=140, train_loss_step=177.0, val_loss=1.9e+3, train_loss_e
Epoch 2:  86%|▊| 6/7 [00:00<00:00,  6.15it/s, loss=165, v_num=140, train_loss_step=161.0, val_loss=1.9e+3, train_loss_e
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 2: 100%|█| 7/7 [00:01<00:00,  4.62it/s, loss=165, v_num=140, 

Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 15: 100%|█| 7/7 [00:01<00:00,  4.92it/s, loss=131, v_num=140, train_loss_step=131.0, val_loss=1.86e+3, train_loss
Epoch 16:  86%|▊| 6/7 [00:00<00:00,  6.32it/s, loss=127, v_num=140, train_loss_step=144.0, val_loss=1.86e+3, train_loss
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 16: 100%|█| 7/7 [00:01<00:00,  5.22it/s, loss=127, v_num=140, train_loss_step=144.0, val_loss=1.86e+3, train_loss
Epoch 17:  86%|▊| 6/7 [00:00<00:00,  6.51it/s, loss=122, v_num=140, train_loss_step=115.0, val_loss=1.86e+3, train_loss
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 17: 100%|█| 7/7 [00:01<00:00,  5.29it/s, loss=122, v_num=140, train_loss_step=115.0, val_los

Epoch 31:  86%|▊| 6/7 [00:01<00:00,  5.99it/s, loss=90.2, v_num=140, train_loss_step=77.50, val_loss=1.81e+3, train_los
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 31: 100%|█| 7/7 [00:01<00:00,  4.96it/s, loss=90.2, v_num=140, train_loss_step=77.50, val_loss=1.81e+3, train_los
Epoch 32:  86%|▊| 6/7 [00:00<00:00,  6.26it/s, loss=89.9, v_num=140, train_loss_step=81.20, val_loss=1.81e+3, train_los
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 32: 100%|█| 7/7 [00:01<00:00,  5.11it/s, loss=89.9, v_num=140, train_loss_step=81.20, val_loss=1.8e+3, train_loss
Epoch 33:  86%|▊| 6/7 [00:00<00:00,  6.50it/s, loss=87.8, v_num=140, train_loss_step=89.90, val_loss=1.8e+3, train_loss
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                     

Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 46: 100%|█| 7/7 [00:01<00:00,  5.06it/s, loss=79.4, v_num=140, train_loss_step=104.0, val_loss=1.77e+3, train_los
Epoch 47:  86%|▊| 6/7 [00:00<00:00,  6.10it/s, loss=77.5, v_num=140, train_loss_step=91.50, val_loss=1.77e+3, train_los
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 47: 100%|█| 7/7 [00:01<00:00,  4.94it/s, loss=77.5, v_num=140, train_loss_step=91.50, val_loss=1.77e+3, train_los
Epoch 48:  86%|▊| 6/7 [00:01<00:00,  5.38it/s, loss=79, v_num=140, train_loss_step=79.40, val_loss=1.77e+3, train_loss_
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 48: 100%|█| 7/7 [00:01<00:00,  4.59it/s, loss=79, v_num=140, train_loss_step=79.40, val_loss

## trend

In [188]:
trainer.checkpoint_callback.best_model_path

# 1
# 3 不小心删了
# 7 'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=17-v2.ckpt'

'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=17-v2.ckpt'

In [189]:
# best_model_path ='D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=13.ckpt'
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [29]:
## 趋势
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions, x = best_tft.predict(val_dataloader,mode="raw", return_x=True)
# 真实值
trend_t=[[17590.5625, 17614.4883, 17650.7695, 17687.3535, 17721.1836, 17752.0820,
         17781.7734]]
x['decoder_target']

# 1 [[17781.7734]]
# 3 [[17721.1836, 17752.0820, 17781.7734]]
# 7 [[17590.5625, 17614.4883, 17650.7695, 17687.3535, 17721.1836, 17752.0820,
#          17781.7734]]

tensor([[4961.4565, 5102.7539, 5149.1890, 2539.7590, 4243.3608, 5048.9487,
         5073.5513]])

In [36]:
# 50%分位数
trend=[[16752.2773, 16507.8223, 16709.7539, 16561.0078, 14986.1904, 15933.0547,
    16418.3398]]
raw_predictions[0][:, :, 3]

# 1 [[18265.6191]]
# 3 [[16270.6729, 16882.4727, 16940.2227]]
# 7 [[16752.2773, 16507.8223, 16709.7539, 16561.0078, 14986.1904, 15933.0547,
#          16418.3398]]

tensor([[4995.1235, 5844.5112, 5795.3169, 5857.1172, 5520.6118, 5817.8086,
         5208.3574]])

In [ ]:
%load_ext tensorboard
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("lightning_logs")
%tensorboard --logdir lightning_logs
# 1 113
# 3 119
# 7 127
# 15 
# 30 
# 60 
# 90 
# cd D:/Jupyter notebook/TFT/lightning_logs
# tensorboard --logdir lightning_logs


## seasonality

In [11]:
trainer.checkpoint_callback.best_model_path
# 1 'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=67.ckpt'
# 3  'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=14-v6.ckpt'
# 7 'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=10-v3.ckpt

'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=10-v3.ckpt'

In [12]:
# best_model_path ='D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=13.ckpt'
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [31]:

# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions, x = best_tft.predict(val_dataloader,mode="raw", return_x=True)
# 真实值
season_t=[[6813.9805, 3752.7571, 4471.0425, 7904.8877, 2244.4563, 2100.9692,
          230.6764]]
x['decoder_target']
# 1 [[230.6764]]
# 3 [[2244.4563, 2100.9692,  230.6764]]
# 7 [[6813.9805, 3752.7571, 4471.0425, 7904.8877, 2244.4563, 2100.9692,
#           230.6764]]

tensor([[4961.4565, 5102.7539, 5149.1890, 2539.7590, 4243.3608, 5048.9487,
         5073.5513]])

In [32]:
# 50%分位数
season=[[4113.8159, 3764.3884, 3863.7390, 3846.8425, 3964.4387, 3843.1409,
         4141.3496]]
raw_predictions[0][:, :, 3]
# 1 [[3974.3286]]
# 3 [[3939.8679, 3936.6040, 4264.2871]]
# 7 
# [[4113.8159, 3764.3884, 3863.7390, 3846.8425, 3964.4387, 3843.1409,
#          4141.3496]]

tensor([[4995.1235, 5844.5112, 5795.3169, 5857.1172, 5520.6118, 5817.8086,
         5208.3574]])

In [ ]:
%load_ext tensorboard
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("lightning_logs")
%tensorboard --logdir lightning_logs
# 1 116
# 3 126
# 7 131
# 15 
# 30 
# 60 
# 90 
# cd D:/Jupyter notebook/TFT/lightning_logs
# tensorboard --logdir lightning_logs


## resid

In [42]:
trainer.checkpoint_callback.best_model_path
# 1 'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=6-v3.ckpt'
# 3 'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=5-v7.ckpt'
# 7 'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=4-v3.ckpt'
# 15 'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=23-v1.ckpt'

'D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=23-v1.ckpt'

In [43]:
# best_model_path ='D:\\Jupyter notebook\\TFT\\saved_models\\best_model_epoch=13.ckpt'
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [44]:
## 趋势
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions, x = best_tft.predict(val_dataloader,mode="raw", return_x=True)
# 真实值
resid_t=[[ 8493.0791,  6935.9995,  5236.9570,  4950.0137,  5191.2085,  5196.5332,
         10220.0273,  9167.5908,  4961.4565,  5102.7539,  5149.1890,  2539.7590,
          4243.3608,  5048.9487,  5073.5513]]
x['decoder_target']
# 1 [[5073.5513]]
# 3 [[4243.3608, 5048.9487, 5073.5513]]
# 7
# [[4961.4565, 5102.7539, 5149.1890, 2539.7590, 4243.3608, 5048.9487,
#          5073.5513]]     
# 15
# [[ 8493.0791,  6935.9995,  5236.9570,  4950.0137,  5191.2085,  5196.5332,
#          10220.0273,  9167.5908,  4961.4565,  5102.7539,  5149.1890,  2539.7590,
#           4243.3608,  5048.9487,  5073.5513]]

tensor([[ 8493.0791,  6935.9995,  5236.9570,  4950.0137,  5191.2085,  5196.5332,
         10220.0273,  9167.5908,  4961.4565,  5102.7539,  5149.1890,  2539.7590,
          4243.3608,  5048.9487,  5073.5513]])

In [45]:
# 50%分位数
resid=[[5398.4741, 5379.2378, 4713.4468, 4839.1294, 5059.2065, 5191.2876,
         5187.0586, 5528.4873, 5407.6030, 4954.3867, 5393.2856, 5137.5254,
         3823.7744, 4239.0381, 5347.8823]]
raw_predictions[0][:, :, 3]
# 1 [[4937.7310]]
# 3 [[4886.5132, 5101.5098, 5329.3799]]
# 7
# [[4995.1235, 5844.5112, 5795.3169, 5857.1172, 5520.6118, 5817.8086,
#          5208.3574]]
# 15
# [[5398.4741, 5379.2378, 4713.4468, 4839.1294, 5059.2065, 5191.2876,
#          5187.0586, 5528.4873, 5407.6030, 4954.3867, 5393.2856, 5137.5254,
#          3823.7744, 4239.0381, 5347.8823]]

tensor([[5398.4741, 5379.2378, 4713.4468, 4839.1294, 5059.2065, 5191.2876,
         5187.0586, 5528.4873, 5407.6030, 4954.3867, 5393.2856, 5137.5254,
         3823.7744, 4239.0381, 5347.8823]])

In [ ]:
%load_ext tensorboard
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("lightning_logs")
%tensorboard --logdir lightning_logs
# 1 118
# 3 124
# 7 132
# 15 
# 30 
# 60 
# 90 
# cd D:/Jupyter notebook/TFT/lightning_logs
# tensorboard --logdir lightning_logs


In [37]:

import numpy as np

# 模型预测值
trend = np.array(trend)
season = np.array(season)
resid = np.array(resid)

# 真实值
trend_t = np.array(trend_t)
season_t = np.array(season_t)
resid_t = np.array(resid_t)

# 计算Tourist和Tourist_t
Tourist = trend + season + resid-9000
Tourist_t = trend_t + season_t + resid_t-9000
Tourist

# 计算MAE、RMSE和MAPE
MAE = np.mean(np.abs(Tourist - Tourist_t))
RMSE = np.sqrt(np.mean((Tourist - Tourist_t)**2))
MAPE = np.mean(np.abs((Tourist - Tourist_t) / Tourist_t)) 
print("tourist:", Tourist)
print("tourist_t:", Tourist_t)
print("MAE:", MAE)
print("RMSE:", RMSE)
print("MAPE:", MAPE)

tourist: [[16861.2167 17116.7219 17368.8098 17264.9675 15471.2409 16594.0042
  16768.0468]]
tourist_t: [[20365.9995 17469.9993 18271.001  19132.0002 15209.0007 15901.9999
  14086.0011]]
MAE: 1466.2248999999986
RMSE: 1868.883412680888
MAPE: 0.08434876507515128


In [ ]:
# 1
# tourist: [[27177.6787]]
# tourist_t: [[23086.0011]]

# 3
# tourist: [[16097.054  16920.5865 17533.8897]]
# tourist_t: [[15209.0007 15901.9999 14086.0011]]

# 7
# tourist: [[16861.2167 17116.7219 17368.8098 17264.9675 15471.2409 16594.0042
#   16768.0468]]
# tourist_t: [[20365.9995 17469.9993 18271.001  19132.0002 15209.0007 15901.9999
#   14086.0011]]

In [92]:
# 1 
# MAE: 4091.6776000000027
# RMSE: 4091.6776000000027
# MAPE: 0.2904

# 3
# MAE: 1784.8428333333322
# RMSE: 2138.0749225530285
# MAPE: 0.12240603418067215

# 7
# MAE: 1466.2248999999986
# RMSE: 1868.883412680888
# MAPE: 0.08434876507515128

In [181]:
trend


array([[16270.6729, 16882.4727, 16940.2227]])

In [182]:
season

array([[3939.8679, 3936.604 , 4264.2871]])

In [183]:
resid

array([[4886.5132, 5101.5098, 5329.3799]])